<a href="https://colab.research.google.com/github/baminro/Dataset-/blob/main/FOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow[and-cuda]

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt # Visualization
import matplotlib.dates as mdates # Formatting dates
import seaborn as sns # Visualization
from sklearn.preprocessing import MinMaxScaler
import torch # Library for implementing Deep Neural Network
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


data=pd.read_excel('/content/drive/MyDrive/copper.xlsx')
data=data.set_index('Time')
data['SMA30Var1'] = data['Var1'].rolling(252).mean()
data['SMA5Var1'] = data['Var1'].rolling(100).mean()
data['SMA5Var1-SMA30Var1']=data['SMA5Var1']-data['SMA30Var1']
data=data[['Var1']]
data.dropna(inplace=True)
data


# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Split the data into train and test sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]


def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length-252):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length+1:i+seq_length+252+1])
    return np.array(X), np.array(y)

seq_length =252  # Number of time steps to look back
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)


import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv1D, Dense,Add, LayerNormalization, MultiHeadAttention, Dropout, GlobalAveragePooling1D
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Transformer Block

def transformer_encoder(inputs, head_size, num_heads, ff_dim,
                        dropout=0, attention_axes=None):

  x = LayerNormalization(epsilon=0.001)(inputs)
  x = MultiHeadAttention(
      key_dim=head_size, num_heads=num_heads, dropout=dropout,
      attention_axes=attention_axes
      )(x, x)
  x = Dropout(dropout)(x)
  res = x + inputs

    # Feed Forward Part
  x = LayerNormalization(epsilon=0.001)(res)
  x = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
  x = Dropout(dropout)(x)
  x = Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
  return x + res

def build_transfromer(head_size,
                      num_heads,
                      ff_dim,
                      num_trans_blocks,
                      mlp_units, dropout=0, mlp_dropout=0) -> tf.keras.Model:

  n_timesteps, n_features, n_outputs = 252, 1, 1
  inputs = tf.keras.Input(shape=(n_timesteps, n_features))
  x = inputs
  for _ in range(num_trans_blocks):
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

  x = GlobalAveragePooling1D(data_format="channels_first")(x)
  for dim in mlp_units:
    x = Dense(dim, activation="relu")(x)

    x = Dropout(mlp_dropout)(x)
  x=keras.layers.Reshape((n_timesteps,1))(x)
  outputs = Dense(1, activation='relu')(x)
 # tf.keras.layers(outputs,(n_timesteps,1))

  return tf.keras.Model(inputs, outputs)

transformer = build_transfromer(head_size=75, num_heads=2, ff_dim=2,
                                num_trans_blocks=2, mlp_units=[252],
                                mlp_dropout=0.20, dropout=0.20,
                              )
transformer.summary()
transformer.compile(
    loss="mse",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["mse"],
)

filepath = 'tf1_mnist_cnn.weights.h5'
save_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=True,
                             mode='auto',  save_freq="epoch",)

history = transformer.fit(X_train, y_train[:,:,0], batch_size=12,
                         epochs=10, validation_data=(X_test, y_test[:,:,0]),
                         verbose=1,shuffle=True, callbacks=save_checkpoint)

# Make predictions
transformer.load_weights('tf1_mnist_cnn.weights.h5')
train_predict = transformer.predict(X_train)
test_predict = transformer.predict(X_test)

# Inverse transform predictions
#train_predict = scaler.inverse_transform(train_predict)
#test_predict = scaler.inverse_transform(test_predict

train_mse=mean_squared_error(y_train[:,:,0], train_predict.squeeze())
test_mse=mean_squared_error(y_test[:,:,0], test_predict.squeeze())

# Evaluate the model (Optional: Calculate RMSE or other metrics)
#train_rmse = math.sqrt(mean_squared_error(y_train, scaler.inverse_transform(train_predict.reshape(-1, 1))))
#test_rmse = math.sqrt(mean_squared_error(y_test, scaler.inverse_transform(test_predict.reshape(-1, 1))))

print(f"Train MSE: {train_mse}")
print(f"Test MSE: {test_mse}")
history

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs=epochs = range(1, len(val_loss) + 1)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Test loss')
plt.title('Training and test loss')
plt.title
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


pre=transformer.predict(test_data[-252:,:].reshape(1, 252, 1))
forecast_period =252
scaler1 = MinMaxScaler()
scaled_data = scaler1.fit_transform(np.array(data['Var1'].values.reshape(-1, 1)))
forecast = scaler1.inverse_transform(np.array(pre).reshape(-1, 1))
forecast

forecast=forecast[:]
vb=scaler.inverse_transform(test_data[-252:,:])[-1][0]
forecast[0]=vb
#vb=vb[-1,-1]
if (forecast[0]/vb)-1>0.05:
    forecast[0]=vb*(1+0.05)
if (forecast[0]/vb)-1<-0.05:
    forecast[0]=vb*(1-0.05)
for i in range(len(forecast)-1):
     if (forecast[i+1]/forecast[i])-1>0.05:
         forecast[i+1]=forecast[i]*(1+0.05)
for i in range(len(forecast)-1):
     if (forecast[i+1]/forecast[i])-1<-0.05:
         forecast[i+1]=forecast[i]*(1-0.05)

import datetime
plt.figure(figsize=(10, 6))
#plt.plot(data.index[-len(test_data):], scaler1.inverse_transform(test_data)[:,0], label='Actual')
plt.plot(pd.date_range(start=data.index[-1]+ datetime.timedelta(days=1), periods=forecast_period, freq='D'), forecast, label='Forecast')
plt.title('Copper Time Series Forecasting (252-day Forecast)')
plt.xlabel('Year')
plt.ylabel('Copper')
plt.legend()
plt.show()
import numpy
import pandas as pd
ret=pd.DataFrame(forecast)/pd.DataFrame(forecast).shift(1)
numpy.std(ret)

pd.DataFrame(forecast).to_excel('/content/drive/MyDrive/forcop.xlsx')
